In [ ]:
from winevt import EventLog

events = []

def handleEvent(action, pContext, event):
    evt = {}
    if hasattr(event, "EventData"):
        if hasattr(event.EventData, "Data"):
            if event.EventData.Data[1].cdata == "Furqan Saeed":
                evt["EventID"] = event.EventID
                for item in event.EventData.Data:
                    evt[item["Name"]] = item.cdata
                events.append(evt)

cb = EventLog.Subscribe("Security", "Event/System[EventID=4663 or EventID=4660]", handleEvent)

In [ ]:
import win32evtlog

num = 0;
hand=win32evtlog.OpenEventLog(None, "Security")
flags= win32evtlog.EVENTLOG_BACKWARDS_READ|win32evtlog.EVENTLOG_SEQUENTIAL_READ

while(1):
    records = win32evtlog.ReadEventLog(hand, flags, 0)
    # print(dir(records[0]))
    if not records:
        break;
    for record in records:
        if record.EventID == 4663:
            print(str(record.EventID) + " " + record.SourceName + " " + str(record.ComputerName) + " " + record.TimeWritten.Format())
            num = num + 1
    
win32evtlog.CloseEventLog(hand)
print(num)

In [1]:
import os
import sys, signal
import win32file
import win32con

def signal_handler(signal, frame):
    print("\nprogram exiting gracefully")
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

ACTIONS = {
    1: "Created",
    2: "Deleted",
    3: "Updated",
    4: "Renamed to ",
    5: ""
}
FILE_LIST_DIRECTORY = 0x0001

path_to_watch = "C:/Test"
hDir = win32file.CreateFile(
    path_to_watch,
    FILE_LIST_DIRECTORY,
    win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE | win32con.FILE_SHARE_DELETE,
    None,
    win32con.OPEN_EXISTING,
    win32con.FILE_FLAG_BACKUP_SEMANTICS,
    None
)

while 1:
    #Right now only directory check is being used and everything else like filecheck is commented
    results = win32file.ReadDirectoryChangesW(
        hDir,
        1024,
        True,
        win32con.FILE_NOTIFY_CHANGE_FILE_NAME |
        win32con.FILE_NOTIFY_CHANGE_DIR_NAME |
        # win32con.FILE_NOTIFY_CHANGE_ATTRIBUTES |
        # win32con.FILE_NOTIFY_CHANGE_SIZE |
        win32con.FILE_NOTIFY_CHANGE_LAST_WRITE,
        # win32con.FILE_NOTIFY_CHANGE_SECURITY,
        None,
        None
    )

    for action, file in results:
        full_filename = os.path.join(path_to_watch, file)
        if action == 4:
            print(full_filename, ACTIONS.get(action, "Unknown"), end='')
        else :
            print(full_filename,ACTIONS.get(action, "Unknown"))

C:/Test\this.txt Updated
